<a href="https://colab.research.google.com/github/srini229/EE5333_tutorials/blob/master/part/Partition.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
!pip install mip

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 15.3/15.3 MB 24.9 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 441.8/441.8 kB 7.4 MB/s eta 0:00:00
  Attempting uninstall: cffi
    Found existing installation: cffi 1.16.0
    Uninstalling cffi-1.16.0:
      Successfully uninstalled cffi-1.16.0


## Partitioning

* Kernighan Lin Algorithm for bi-partitioning ($V'$) :
  + $G=(V,E)$
  + $A$, $B$ $\subset V$
  + $A \cup B = V$
  + $A\cap B = ∅$
  + $|A| = |B| = \dfrac{|V|}{2}$
  + Flowchart:

    <img src="https://raw.githubusercontent.com/srini229/EE5333_tutorials/master/part/fig/KL_flowchart.jpg" width=698 height=612 />


In [1]:
# Vertex class to hold the partition index, neighbours, EA, EB and D values
class Vertex:
  def __init__(self, i, part):
    self._id = i
    self._nbrs = []
    self._part = part
    self._ea = 0
    self._eb = 0
    self._d  = 0
  def reset(self, part):
    (self._part, self._ea, self._eb, self._d) = (part, 0, 0, 0)
  def __str__(self):
    return '(' + str(self._id) + ',' + str(self._part) + ',' + str(self._ea) + ',' + str(self._eb) + ',' + str(self._d) + ',' + str([i._id for i in self._nbrs]) + ')'
  def __repr__(self):
    return str(self)

# clear the partition, EA, EB and D values
# do this at the beginning of every iteration
def reset(V, A, B):
  for j in range(2):
    partition = A if (0 == j) else B
    for i in partition:
      V[i].reset(j)
  for v in V:
    assert(v._part == 0 or v._part == 1)
    for n in v._nbrs:
      if n._part == 0:
        v._ea += 1
      else:
        v._eb += 1
  for v in V:
    v._d = (v._ea - v._eb) if (v._part == 1) else (v._eb - v._ea)

# Choose the pair whose swap has the maximum gain in number of cuts
def findMaxGain(V, Ap, Bp, E):
  (amax, bmax, gmax) = (-1, -1, -2 * len(E) - 1)
  for a in Ap:
    for b in Bp:
      g = V[a]._d + V[b]._d - (2 if (min(a,b), max(a,b)) in E else 0)
      if gmax < g:
        (amax, bmax, gmax) = (a, b, g)
  assert(amax >= 0 and bmax >= 0)
  return (amax, bmax, gmax)

# update the E and D for only the affected neighbours of a and b
def updateED(V, a, b):
  V[a]._part = 1
  V[b]._part = 0
  for i in [a,b]:
    for n in V[i]._nbrs:
      if i == a:
        n._ea -= 1
        n._eb += 1
      else:
        n._ea += 1
        n._eb -= 1
      n._d = (n._ea - n._eb) if (n._part == 1) else (n._eb - n._ea)

# N is the number of vertices; vertices are {0, 1,... N-1}
# E is the list of edges
# E : list of edges; edge = unordered pair of vertices
# Return value : two sets A, B and the count of number of cuts
def KLPart(N, E):
  if N%2: N+= 1 # make N even if its odd by adding a single no-neighbour vertex
  V = [Vertex(i, -1) for i in range(N)]
  for e in E:
    if e[0] > e[1]: e = (e[1], e[0])
    else: e = (e[0], e[1])
  E = set(E)
  for e in E:
    V[e[0]]._nbrs.append(V[e[1]])
    V[e[1]]._nbrs.append(V[e[0]])
  import random
  Vc = V[:]
  partLen = N//2
  random.shuffle(Vc) # randomly initialize A and B
  A = {Vc[i]._id for i in range(partLen)}
  B = {Vc[i]._id for i in range(partLen, N)}

  maxGain, prevGain = 1, 1
  while maxGain >= 0:
    Ap, Bp= A.copy(), B.copy()
    reset(V, A, B)
    G, S = [], []
    for p in range(partLen):
      (a, b, g) = findMaxGain(V, Ap, Bp, E)
      updateED(V, a, b)
      Ap.remove(a)
      Bp.remove(b)
      G.append(g)
      S.append((a, b))
    for i in range(1, len(G)):
      G[i] += G[i-1]
    prevGain = maxGain
    maxGain = max(G)
    maxIndex = G.index(maxGain)
    if maxGain > 0:
      for (a, b) in S[0:maxIndex + 1]:
        A.remove(a)
        B.remove(b)
        A.add(b)
        B.add(a)
    if prevGain == 0 and maxGain == 0:
      break

  cut = 0
  for a in A:
    for b in B:
      if (min(a, b), max(a,b)) in E:
        cut += 1
  return (A, B, cut)

In [2]:
print(KLPart(8, [(0,1), (0,4), (0,5), (1,4), (1,5), (4,5), (2,3), (2,6), (2,7), (3,6), (3,7), (6,7), (2,5)]))

({0, 1, 4, 5}, {2, 3, 6, 7}, 1)


## Bipartitioning using ILP
+ $x_v$ is the indicator variable for $v$ being in $A$
+ $x_{u,v}$ is the indicator variable for $(u,v)\in E$ being cut
+ <ul>
$\begin{align}
        x_{u,v} = x_u \oplus x_v
\end{align}$
</ul>

+ Objective: $\min\limits_{x_v, x_{u,v}} \sum\limits_{(u,v)\in E}x_{u,v}$
+ Subject to constraints:
<ul>
$\begin{align}
\sum_{v\in V} x_v&=\frac{|V|}{2}\\
x_u - x_v &\leq x_{u,v}, &\forall (u,v) \in E\\
x_v - x_u &\leq x_{u,v}, &\forall (u,v) \in E\\
x_u + x_v &\geq x_{u,v}, &\forall (u,v) \in E\\
x_u + x_v + x_{u,v} &\leq 2, &\forall (u,v) \in E\\
x_v &\in \{0, 1\}, &\forall v \in V\\
x_{u,v} &\in \{0, 1\}, &\forall (u,v) \in E
\end{align}$
</ul>


In [7]:
def bipartition(N, E):
  import mip
  model = mip.Model("Bi-partition")
  x = [model.add_var(f"x{u}", var_type = mip.BINARY) for u in range(N)]
  x_uv = [model.add_var(f"x{u}_{v}", var_type = mip.BINARY) for u,v in E]
  model.verbose = 0
  model.objective = mip.minimize(mip.xsum(x_uv))
  model += (mip.xsum(x) == N//2)
  for e, (u,v) in enumerate(E):
  # xor constraints
    model += (x[u] - x[v] <= x_uv[e])
    model += (x[v] - x[u] <= x_uv[e])
    model += (x[u] + x[v] >= x_uv[e])
    model += (x[u] + x[v] + x_uv[e] <= 2)

  model.write("bipartition.lp")
  model.optimize()
  if model.status == mip.OptimizationStatus.OPTIMAL:
    A = [i for i in range(N) if x[i].x >= 0.9]
    B = [i for i in range(N) if x[i].x < 0.9]
    return (A, B, model.objective.x)
  return None



In [8]:
N = 8
E = [(0,1), (0,4), (0,5), (1,4), (1,5), (1,6),
     (2,3), (2,6), (2,7), (3,6), (3,7), (4,5),
     (6,7)]
print(bipartition(N, E))

([0, 1, 4, 5], [2, 3, 6, 7], 1.0)
